In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'seaborn'

# Збір даних

Перш за все, потрібно зібрати дані з акселерометра мобільного телефону.
<center><img src="image.png"/></center>


In [ ]:
from pathlib import Path
activities = ['idle', 'running', 'stairs', 'walking']
data_collections = []

for index, activity in enumerate(activities, start=1):
    activity_path = Path('data') / activity  
    csv_files = list(activity_path.glob('*.csv'))  
    df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)
    df['activity'] = activity
    data_collections.append(df)

data = pd.concat(data_collections, ignore_index=True)
data

# Тренування на вихідних нормалізованих данних

В якості характеристик візьмемо показники з акселерометра

In [ ]:
features = ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']

## Нормалізація даних

Тренування на ненормаліованих даних займає чимало часу. Вхідні дані нормалізуються так, щоб їхнє середнє значення дорівнювало нулю, а стандартне відхилення - одиниці. 

In [ ]:
scaler = StandardScaler()

data_to_normalize = data[features]

# Нормалізація даних
normalized_data = scaler.fit_transform(data_to_normalize)

# Заміна вихідних даних нормалізованими даними
ndata = data.copy()
ndata[features] = normalized_data
ndata

## Розділяємо дані

Розділяємо датасет на навчальний і тестовий набори

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data[features], data["activity"],
    test_size=0.3,
    stratify=ndata["activity"],
)

## Навчаємо за допомогою алгоритму SVM

In [ ]:
%%time

model_svm = SVC()
model_svm.fit(X_train, y_train)

## Навчаємо за допомогою алгоритму випадкового лісу

In [ ]:
%%time

model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train, y_train)

## Порівнюємо точність результатів

Визначаємо точність моделей на тестовому наборі та порівнюємо

In [ ]:
score_svm = model_svm.score(X_test, y_test)
score_rf = model_rf.score(X_test, y_test)

print(f"Точність моделі SVM: {score_svm:.3f}")
print(f"Точність моделі випадкового лісу: {score_rf:.3f}")

## Пргнозування

In [ ]:
y_predicted_svm = model_svm.predict(X_test)
y_predicted_rf = model_rf.predict(X_test)

Побудуємо матрицю помилок (`confusion matrix`). 

Матрицю помилок - це таблиця, яку використовують для оцінювання продуктивності класифікаційної моделі. Вона дає змогу порівняти фактичні та передбачені класифікатором мітки для кожного об'єкта в тестовій вибірці.

In [ ]:
conf_matrix_svm = confusion_matrix(y_test, y_predicted_svm)
conf_matrix_rf = confusion_matrix(y_test, y_predicted_rf)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

classifiers = ["SVM", "Random Forest"]
confusion_matrices = [conf_matrix_svm, conf_matrix_rf]

for i, classifier in enumerate(classifiers):
    sns.heatmap(
        confusion_matrices[i],
        annot=True,
        fmt='d',
        cmap='Reds',
        xticklabels=activities,
        yticklabels=activities,
        ax=axes[i],
    )
    axes[i].set_title(f"Confusion Matrix for {classifier} Classifier")
    axes[i].set_xlabel("Predicted activity")
    axes[i].set_ylabel("True activity")

plt.tight_layout()

plt.show()

# Тренування на підготовленних даних

## Вибираємо ознаки

В [статті](https://www.sciencedirect.com/science/article/pii/S1877050916322153) [1] пропонується обрати наступні часові ознаки (таблиця 3). Як пишуть автори, їхні результати показують, що найбільш репрезентативні ознаки отримані з часового представлення сигналу акселерометра. Ознаки, витягнуті з сигналів по осях $x$ і $y$ є більш релевантними, ніж ознаки по осі $z$ при визначенні активності. 

| Name                          | Axis |
| ----------------------------- | ---- |
| Maximum Value                 |   x  |
| Minimum Value                 |   x  |
| Entropy                       |   x  |
| Interquartile Range           |   x  |
| Maximum Value                 |   y  |
| Index of Minimum Value        |   y  |
| Mean of Absolute Deviation    |   y  |
| Median                        |   y  |
| Skewness                      |   y  |
| Standard Deviation            |   y  |
| Root Mean Square Error        |   y  |
| Skewness                      |   z  |

## Створення необхідних функцій

In [ ]:
def iqr(data):
    """Calculates the interquartile range (IQR) of a dataset.

    The IQR is the difference between the 75th and 25th percentiles of the data,
    and is a measure of the spread of the middle 50% of the data.

    Args:
        data: A NumPy array or Pandas Series containing the data.

    Returns:
        The IQR of the data.
    """
    sorted_data = np.sort(data)
    
    Q1 = np.percentile(sorted_data, 25)
    Q3 = np.percentile(sorted_data, 75)
    
    IQR = Q3 - Q1
    
    return IQR
  
def argmin(data):
    """Returns the index of the minimum value in a dataset.

    Args:
        data: A NumPy array or Pandas Series containing the data.

    Returns:
        The index of the minimum value in the data.
    """
    return np.argmin(data)

def entropy(column_data):
    """Calculates the entropy of a column of data.

    Entropy is a measure of uncertainty or randomness in a set of data. It is calculated as follows:

    ```
    H = -sum(pk * log2(pk))
    ```

    where pk is the probability of each value in the data set.

    Args:
        column_data: A NumPy array or Pandas Series containing the data.

    Returns:
        The entropy of the data.
    """
    
    hist, bins = np.histogram(column_data, bins='auto')
    probs = hist / len(column_data)

    probs = probs[probs > 0]

    entropy = -np.sum(probs * np.log2(probs))

    return entropy

def mad(data):
    """Calculates the median absolute deviation (MAD) of a dataset.

    The MAD is a measure of the spread of the data, and is calculated as the median of the absolute deviations from the median. It is a more robust measure of spread than the standard deviation, as it is less sensitive to outliers.

    Args:
        data: A NumPy array or Pandas Series containing the data.

    Returns:
        The MAD of the data.
    """
    mean = np.mean(data)
    
    absolute_deviations = np.abs(data - mean)
    
    mad = np.mean(absolute_deviations)
    
    return mad

def rmse(data):
    """Calculates the root mean square error (RMSE) of a dataset.

    The RMSE is a measure of the average difference between the actual values in the dataset and the predicted values. It is calculated as follows:

    ```
    RMSE = sqrt(mean((y_true - y_pred)**2))
    

    where:

    * `y_true` is the actual values in the dataset
    * `y_pred` is the predicted values

    The RMSE is a good measure of fit for regression models, and is often used to evaluate the performance of a model on a held-out test set.

    Args:
        data: A NumPy array or Pandas Series containing the data.

    Returns:
        The RMSE of the data.
    """
    
    mean = np.mean(data)
    
    squared_errors = [(x - mean) ** 2 for x in data]
    
    rmse = np.sqrt(np.mean(squared_errors))
    
    return rmse

## Створення датасету на основі ознак, вказанних в статті

In [ ]:
axis_aggregations = {
    'accelerometer_X': ["max", "min", entropy, iqr],
    'accelerometer_Y': ["max", argmin, mad, "median", "skew", "std", rmse],
    'accelerometer_Z': ["skew"]
}

data_tdf = pd.DataFrame()


for axis, functions in axis_aggregations.items():
    axis_data = ndata.groupby(["activity", ndata.index // 30])[axis].agg(functions)
    
    multiindex = pd.MultiIndex.from_product([[axis], axis_data.columns])
    
    axis_data.columns = multiindex
    data_tdf = pd.concat([data_tdf, axis_data], axis=1)

data_tdf.reset_index(level=0, inplace=True) 

In [ ]:
# Утворений датасет на основі ознак таблиці 3 статті [1]

data_tdf

## Розділяємо датасет на навчальний і тестовий набори

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    data_tdf[features], data_tdf["activity"],
    test_size=0.3,
    stratify=data_tdf["activity"],
)

## Навчаємо за домомогою алгоритму `SVN`

In [ ]:
%%time

model_svm = SVC()
model_svm.fit(X_train, y_train)

## Навчаємо за домомогою алгоритму `random forest`

In [ ]:
%%time

model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train, y_train)

## Визначаємо точність моделей на тестовому наборі

In [ ]:
score_svm = model_svm.score(X_test, y_test)
score_rf = model_rf.score(X_test, y_test)

print(f"Точність моделі SVM: {score_svm:.3f}")
print(f"Точність моделі випадкового лісу: {score_rf:.3f}")

## Пргнозування

In [ ]:
y_predicted_svm = model_svm.predict(X_test)
y_predicted_rf = model_rf.predict(X_test)

Побудуємо матрицю помилок `confusion matrix`

In [ ]:
conf_matrix_svm = confusion_matrix(y_test, y_predicted_svm)
conf_matrix_rf = confusion_matrix(y_test, y_predicted_rf)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

classifiers = ["SVM", "Random Forest"]
confusion_matrices = [conf_matrix_svm, conf_matrix_rf]

for i, classifier in enumerate(classifiers):
    sns.heatmap(
        confusion_matrices[i],
        annot=True,
        fmt='d',
        cmap='Greens',
        xticklabels=activities,
        yticklabels=activities,
        ax=axes[i],
    )
    axes[i].set_title(f"Confusion Matrix for {classifier} Classifier")
    axes[i].set_xlabel("Predicted activity")
    axes[i].set_ylabel("True activity")

plt.tight_layout()

plt.show()

- Як видно, класифікатор `SVM` на тестових даних помилився у всіх 49 випадках, класифікувавши ходіння по сходах (`stairs`) як прогулянку (`walking`). Тобто, зовсім не розпізнає ходіння по сходах.

- Класифікатор `ramdom forest` на тестових даних помилився лише в 5 випадках, класифікувавши ходіння по сходах (`stairs`) як прогулянку (`walking`). 

Слід сказати. що набір даних для `stairs` був найменшим з усіх інших. Тому тут бажано було б набрати більше даних, тоді може `SVM` дав би кращі результати.

# Результати та висновки


1. Для навчання алгоритму `SVN` на __вихідних ознаках__ (показах акселерометра по трьох осях) потрібно близько 20 хвилин на стандартному комп'ютері. Навчання на вихідних ознаках (показах акселерометра по трьох осях) спричиняє довге навчання, оскільки алгоритм `SVN` має велику кількість параметрів, які потрібно оптимізувати. Це пов'язано з тим, що алгоритм `SVN` намагається знайти оптимальну підмножину ознак, яка найкраще класифікує дані. У випадку з даними про рух людини, які мають багато шуму і невизначеності, це може бути дуже складним завданням.

2. Навчання на __нормалізованих ознаках__ акселерометра по трьох осях зменшує час навчання алгоритму `SVN` близько 4 хвилин. Навчання на нормалізованих величинах акселерометра по трьох осях дає значний виграш при навчання по методу `SVN`. Нормалізація показів акселерометра по трьох осях усуває масштабування, яке може призвести до переваги одних ознак над іншими. Це також робить дані більш однорідними, що полегшує задачу навчання алгоритму `SVN`.
    - час розрахунку близько 13 секунд;
    - точність близька до 0.89.


3. Навчання за методом випадкового лісу показує кращу точність та набагато менший час навчання, оскільки це більш стійкий до шуму і невизначеності алгоритм. Метод випадкового лісу генерує декілька дерев рішень, і кожен з них голосує за класифікацію. Це дозволяє алгоритму випадкового лісу бути більш точним, ніж алгоритм `SVN`, який генерує лише одне дерево рішень. Крім того, метод випадкового лісу має набагато меншу кількість параметрів, ніж алгоритм `SVN`, що також сприяє його більш швидкому навчанню.
    - час розрахунку близько 13 секунд;
    - точність близька до 1.00.

4. Навчання, де в якості ознак взяті статистичні ознаки запропоновані в [статті](https://www.sciencedirect.com/science/article/pii/S1877050916322153) [1], отримані по набору з 30 показників акселерометра для кожної осі $x$, $y$ та $z$ показує:
    - точність моделі `SVM` значно підвищилась до значення 0.975, крім того час навчання значно зменшився і становить 46.9 ms.
    - точність моделі `random forest` також підвищилась в порівнянні з навчанням на вихідних нормалізованих судячи з матриці помилок, також час навчання становить 766 ms, що значно менше ніж для навчання на вихідних нормалізованих даних, але натомість він на порядок вище в порявнянні з моделлю `SVN`.

Таблиця з результатами

<center>
<table>
<caption>Таблиця. Результати</caption>
<thead>
  <tr>
    <th>Тип ознак</th>
    <th>Алгоритм</th>
    <th>Час розрахунку</th>
    <th>Точність</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td rowspan="2">вихідні нормалізовані</td>
    <td>SVN</td>
    <td>240 s</td>
    <td>0.89</td>
  </tr>
  <tr>
    <td>Random Forest</td>
    <td>13 s</td>
    <td>~ 1.00</td>
  </tr>
  <tr>
    <td rowspan="2">статистичні дані</td>
    <td>SVN</td>
    <td>46.9 ms</td>
    <td>0.98</td>
  </tr>
  <tr>
    <td>Random Forest</td>
    <td>766 ms</td>
    <td>~ 1.00</td>
  </tr>
</tbody>
</table>
</center>

Згідно таблиці, можна зробити висновок, що найкращі результати дає в сенсі час/чточність дає алгоритм `random forest`, який тренувався не на вихідних, а на статистичних ознаках. Ці висновки також підтверджуються аналізом матриці помилок, показує, що класифікатор `random forest` помиляється значно менше, ніж `SVM`. Використання статистичних даних, порівняно з сирими підвищує точність класифікації та знижує час навчання.